In [ ]:
from Src.pre_processing import load_data, split, to_tensor, to_labels 
from Src.Sentan_Model import Sentan_simple
from Src.Dias_Model import Dias_Model
from Src.pre_processing import split, to_tensor

import torch
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch import nn

from torchmetrics import F1Score, Recall, Precision, Accuracy
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### kfold training

In [ ]:
def train_kfold(model_builder, X, y, n_folds, optimizer_builder, lossFn, device='cpu', lr=1e-3, epochs=100):
    
    f1 = F1Score(num_classes=8)
    recall = Recall(average='macro', num_classes=8)
    precision = Precision(average='macro', num_classes=8)
    accuracy = Accuracy()

    # K-folds
    for train_idx, test_idx in split(X, y, splits=n_folds):


        # Activación del entorno
        wandb.init(project="Clasificador de sentimientos", entity="panas")

        model = model_builder().to(device)
        optimizer = optimizer_builder(
            model.parameters(), lr=0.001, weight_decay=0.03)

        # Fold Data
        x_train = X[train_idx]
        y_train = y[train_idx]

        x_test = X[test_idx]
        y_test = y[test_idx]

        x_train, y_train = to_tensor(x_train, y_train)
        x_test, y_test = to_tensor(x_test, y_test)

        # Train loop
        model.train()
        for epoch in range(0, epochs):

            # Training
            x_train = x_train.to(device)
            y_train = y_train.to(device)

            optimizer.zero_grad()

            pred = model(x_train)

            loss = lossFn(pred, y_train)
            loss.backward()
            optimizer.step()

            train_accuracy = accuracy(pred, y_train)

        # Validation
            with torch.no_grad():

                x_test = x_test.to(device)
                y_test = y_test.to(device)
                pred = model(x_test)

                wandb.log({'Train Accuracy': train_accuracy, 'Train Loss': loss, 'Validation Accuracy': accuracy(
                    pred, y_test), 'F1 Score': f1(pred, y_test), 'Recall': recall(pred, y_test), 'Precision': precision(pred, y_test)})

## Data

In [ ]:
data_path = 'Data/data.pkl'
X, y, labels = load_data(data_path)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Configuración W&B

In [ ]:
epochs = 200
learning_rate = 1e-3
batch_size = X_train.shape[0]


# Configuración de entorno weights and biases
wandb.config = {
    "model-name": 'Dias model',
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size

}

## Training KFold

In [ ]:
# Entrenamiento de n modelos
models = [Dias_Model]
folds = 10

# Model loop
for model in models:
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam

    train_kfold(model, X, y, folds,  optimizer, loss_fn, device, learning_rate, epochs=epochs)